<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/hojichar%E5%90%8D%E5%AD%97%E3%83%95%E3%82%A3%E3%83%AB%E3%82%BF%E3%83%BC_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install hojichar==0.9.0 -U
!pip show hojichar
import re
import hojichar
import os
import json
import random
from multiprocessing import cpu_count

# ダミーデータの生成
num_samples = 100000  # 生成するサンプル数
dummy_texts = ["こんにちは山本君、これはテストデータです。佐藤さん", "田中くん、ダミーテキストのサンプルです。", "HojiCharを使用して前処理を行います。"]
with open("your_text.jsonl", "w") as f:
    for _ in range(num_samples):
        text = random.choice(dummy_texts)
        f.write(json.dumps({"text": text}) + "\n")

# 敬称別漢字除去フィルター
class RemoveKanjiBeforeHonorific(hojichar.core.filter_interface.Filter):
    """
    「さん」や「くん」などの敬称の直前にある漢字を削除するフィルター
    """

    def apply(self, document):
        """
        処理対象のテキストに対して、敬称の直前の漢字を削除する処理を実行します。

        Args:
            document (hojichar.document.Document): 処理対象のドキュメント

        Returns:
            hojichar.document.Document: 処理後のドキュメント
        """
        text = document.text

        # 敬称別漢字除去パターン
        patterns = {
            "さん": r"(?P<kanji>[\u4e00-\u9fa5]+)(?P<honorific>さん)",
            "くん": r"(?P<kanji>[\u4e00-\u9fa5]+)(?P<honorific>くん)",
            "君": r"(?P<kanji>[\u4e00-\u9fa5]+)(?P<honorific>君)",
        }

        # 各パターンに対して置換処理を実行
        for honorific, pattern in patterns.items():
            text = re.sub(pattern, r"\g<honorific>", text)

        document.text = text
        return document

# Composeオブジェクトの定義
cleaner = hojichar.Compose([
    hojichar.document_filters.JSONLoader(),
    hojichar.document_filters.DocumentNormalizer(),
    RemoveKanjiBeforeHonorific(),  # CustomFilterから変更
    hojichar.document_filters.JSONDumper(),
])

# 大量のJSON Linesファイルを並列に処理
input_file = "your_text.jsonl"
input_doc_iter = (hojichar.Document(line) for line in open(input_file))

num_cores = cpu_count()
with hojichar.Parallel(cleaner, num_jobs=num_cores) as pfilter:
    out_doc_iter = pfilter.imap_apply(input_doc_iter)
    with open("your_processed_text.jsonl", "w") as fp:
        for doc in out_doc_iter:
            fp.write(doc.text + "\n")

# your_processed_text.jsonlから一部のデータを取り出して表示
num_display = 100  # 表示するデータの数
with open("your_processed_text.jsonl", "r") as f:
    for i, line in enumerate(f):
        if i >= num_display:
            break
        print(line.strip())

Name: hojichar
Version: 0.9.0
Summary: Text preprocessing management system.
Home-page: https://github.com/HojiChar/HojiChar
Author: kenta.shinzato
Author-email: hoppiece@gmail.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: mmh3, numpy, tqdm
Required-by: 
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "こんにちは君、これはテストデータです。さん"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "こんにちは君、これはテストデータです。さん"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "こんにちは君、これはテストデータです。さん"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "こんにちは君、これはテストデータです。さん"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "くん、ダミーテキストのサンプルです。"}
{"text": "こんにちは君、これはテストデータです。さん"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "くん、